## Code Modules & Functions

In [0]:
import numpy as np,pandas as pd,pylab as pl
import h5py,torch,urllib,zipfile
from torchvision.datasets import MNIST as tmnist
from torchvision import transforms
from torch.utils.data import DataLoader as tdl
from torch.utils.data import Dataset as tds
import torch.nn.functional as tnnf
from sklearn.datasets import make_classification
from IPython.core.magic import register_line_magic
dev=torch.device("cuda:0" if torch.cuda.is_available() 
                 else "cpu")

In [0]:
def model_acc(model,data_loader,num_features):
    correct_preds,num_examples=0,0    
    for features,targets in data_loader:
        features=features.view(-1,num_features).to(dev)
        targets=targets.to(dev)
        logits,probs=model(features)
        _,pred_labels=torch.max(probs,1)
        num_examples+=targets.size(0)
        correct_preds+=(pred_labels==targets).sum()        
    return correct_preds.float()/num_examples*100

In [0]:
@register_line_magic
def print_acc(t):
    if t=='test':
        print('Test accuracy: %.4f%%'%\
        (model_acc(model,test_loader,num_features)))
    if t=='train':
        print('Train accuracy: %.4f%%'%\
        (model_acc(model,train_loader,num_features)))
@register_line_magic
def print_acc2(t):
    if t=='test':
        print('Test accuracy: %.4f%%'%\
        (model_acc(model,test_loader2,num_features2)))
    if t=='train':
        print('Train accuracy: %.4f%%'%\
        (model_acc(model,train_loader2,num_features2)))

In [0]:
@register_line_magic
def train_run(epochs):
    epochs=int(epochs)
    for epoch in range(epochs):
        for batch_ids,(features,targets) in enumerate(train_loader):        
            features=features.view(-1,num_features).to(dev)
            targets=targets.to(dev)
            logits,probs=model(features)
            cost=tnnf.cross_entropy(logits,targets)
            optimizer.zero_grad(); cost.backward()
            optimizer.step()
            if not batch_ids%300:
                print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                       %(epoch+1,epochs,batch_ids, 
                         len(train)//batch_size,cost))           
        with torch.set_grad_enabled(False):
            print('Epoch: %03d/%03d train accuracy: %.2f%%'%\
                  (epoch+1,epochs,model_acc(model,train_loader,
                                            num_features)))
@register_line_magic
def train_run2(epochs):
    epochs=int(epochs)
    for epoch in range(epochs):
        for batch_ids,(features,targets) in enumerate(train_loader2):        
            features=features.view(-1,num_features2).to(dev)
            targets=targets.to(dev)
            logits,probs=model(features)
            cost=tnnf.cross_entropy(logits,targets.long())
            optimizer.zero_grad(); cost.backward()
            optimizer.step()
            if not batch_ids%300:
                print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                       %(epoch+1,epochs,batch_ids, 
                         len(train2)//batch_size2,cost))           
        with torch.set_grad_enabled(False):
            print('Epoch: %03d/%03d train accuracy: %.2f%%'%\
                  (epoch+1,epochs,model_acc(model,train_loader2,
                                            num_features2)))

## Data

In [12]:
random_seed=1; batch_size=64
train=tmnist(root='data',train=True,download=True,
            transform=transforms.ToTensor())
test=tmnist(root='data',train=False, 
            transform=transforms.ToTensor())
train_loader=tdl(dataset=train,shuffle=True, 
                 batch_size=batch_size)
test_loader=tdl(dataset=test,shuffle=False, 
                batch_size=batch_size)
for images,labels in train_loader:  
    print('Image dimensions: %s'%str(images.shape))
    print('Label dimensions: %s'%str(labels.shape))
    break

Image dimensions: torch.Size([64, 1, 28, 28])
Label dimensions: torch.Size([64])


In [13]:
fpath='https://olgabelitskaya.github.io/'
zf='LetterColorImages_123.h5.zip'
input_file=urllib.request.urlopen(fpath+zf)
output_file=open(zf,'wb'); 
output_file.write(input_file.read())
output_file.close(); input_file.close()
zipf=zipfile.ZipFile(zf,'r')
zipf.extractall(''); zipf.close()
f=h5py.File(zf[:-4],'r')
keys=list(f.keys()); print(keys)
X=np.array(f[keys[1]],dtype='float32')/255
y=np.array(f[keys[2]],dtype='int32')-1
N=len(y); n=int(.2*N); batch_size=16
shuffle_ids=np.arange(N)
np.random.RandomState(23).shuffle(shuffle_ids)
X,y=X[shuffle_ids],y[shuffle_ids]
X_test,X_train=X[:n],X[n:]
y_test,y_train=y[:n],y[n:]
X_train.shape,y_train.shape

['backgrounds', 'images', 'labels']


((11352, 32, 32, 3), (11352,))

In [14]:
random_seed=1; batch_size2=64
class TData(tds):
    def __init__(self,X,y):   
        self.X=torch.tensor(X,dtype=torch.float32)
        self.y=torch.tensor(y,dtype=torch.int32)
    def __getitem__(self,index):
        train_img,train_lbl=self.X[index],self.y[index]
        return train_img,train_lbl
    def __len__(self):
        return self.y.shape[0]
train2=TData(X_train,y_train)
test2=TData(X_test,y_test)
train_loader2=tdl(dataset=train2,batch_size=batch_size2,shuffle=True)
test_loader2=tdl(dataset=test2,batch_size=batch_size2,shuffle=False)
for images,labels in train_loader2:  
    print('Image dimensions: %s'%str(images.shape))
    print('Label dimensions: %s'%str(labels.shape))
    break

Image dimensions: torch.Size([64, 32, 32, 3])
Label dimensions: torch.Size([64])


## MLP with BatchNormalization

In [0]:
num_features=784; num_classes=10
hidden1=512; hidden2=256; hidden3=128
class MLPN(torch.nn.Module):
    def __init__(self,num_features,num_classes):
        super(MLPN,self).__init__()
        self.linear1=torch.nn.Linear(num_features,hidden1)
        self.linear1.weight.detach().normal_(0.,.1)
        self.linear1.bias.detach().zero_()
        self.linear1bn=torch.nn.BatchNorm1d(hidden1)
        self.linear2=torch.nn.Linear(hidden1,hidden2)
        self.linear2.weight.detach().normal_(0.,.1)
        self.linear2.bias.detach().zero_()
        self.linear2bn=torch.nn.BatchNorm1d(hidden2)
        self.linear3=torch.nn.Linear(hidden2,hidden3)
        self.linear3.weight.detach().normal_(0.,.1)
        self.linear3.bias.detach().zero_()
        self.linear3bn=torch.nn.BatchNorm1d(hidden3)
        self.linear_out=torch.nn.Linear(hidden3,num_classes)
        self.linear_out.weight.detach().normal_(0.,.1)
        self.linear_out.bias.detach().zero_() 
    def forward(self,x):
        y=self.linear1(x); y=self.linear1bn(y)
        y=tnnf.relu(y)
        y=self.linear2(y); y=self.linear2bn(y)
        y=tnnf.relu(y)
        y=self.linear3(y); y=self.linear3bn(y)
        y=tnnf.relu(y)
        logits=self.linear_out(y)
        probs=tnnf.log_softmax(logits,dim=1)
        return logits,probs   
torch.manual_seed(random_seed)
model=MLPN(num_features=num_features,
           num_classes=num_classes)
model=model.to(dev); learning_rate=.01
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [16]:
%train_run 20

Epoch: 001/020 | Batch 000/3750 | Cost: 2.6883
Epoch: 001/020 | Batch 300/3750 | Cost: 0.8777
Epoch: 001/020 | Batch 600/3750 | Cost: 0.6617
Epoch: 001/020 | Batch 900/3750 | Cost: 0.4860
Epoch: 001/020 train accuracy: 88.61%
Epoch: 002/020 | Batch 000/3750 | Cost: 0.5334
Epoch: 002/020 | Batch 300/3750 | Cost: 0.2812
Epoch: 002/020 | Batch 600/3750 | Cost: 0.2545
Epoch: 002/020 | Batch 900/3750 | Cost: 0.3162
Epoch: 002/020 train accuracy: 91.70%
Epoch: 003/020 | Batch 000/3750 | Cost: 0.2939
Epoch: 003/020 | Batch 300/3750 | Cost: 0.4081
Epoch: 003/020 | Batch 600/3750 | Cost: 0.3482
Epoch: 003/020 | Batch 900/3750 | Cost: 0.2484
Epoch: 003/020 train accuracy: 93.36%
Epoch: 004/020 | Batch 000/3750 | Cost: 0.1364
Epoch: 004/020 | Batch 300/3750 | Cost: 0.2304
Epoch: 004/020 | Batch 600/3750 | Cost: 0.1658
Epoch: 004/020 | Batch 900/3750 | Cost: 0.1506
Epoch: 004/020 train accuracy: 94.25%
Epoch: 005/020 | Batch 000/3750 | Cost: 0.1944
Epoch: 005/020 | Batch 300/3750 | Cost: 0.1668
Ep

In [17]:
%print_acc train
%print_acc test

Train accuracy: 98.4433%
Test accuracy: 96.6800%


In [0]:
num_features2=32*32*3; num_classes2=33
hidden1=1024; hidden2=256; hidden3=256
class MLPN2(torch.nn.Module):
    def __init__(self,num_features,num_classes):
        super(MLPN2,self).__init__()
        self.linear1=torch.nn.Linear(num_features,hidden1)
        self.linear1.weight.detach().normal_(0.,.1)
        self.linear1.bias.detach().zero_()
        self.linear1bn=torch.nn.BatchNorm1d(hidden1)
        self.linear2=torch.nn.Linear(hidden1,hidden2)
        self.linear2.weight.detach().normal_(0.,.1)
        self.linear2.bias.detach().zero_()
        self.linear2bn=torch.nn.BatchNorm1d(hidden2)
        self.linear3=torch.nn.Linear(hidden2,hidden3)
        self.linear3.weight.detach().normal_(0.,.1)
        self.linear3.bias.detach().zero_()
        self.linear3bn=torch.nn.BatchNorm1d(hidden3)
        self.linear_out=torch.nn.Linear(hidden3,num_classes)
        self.linear_out.weight.detach().normal_(0.,.1)
        self.linear_out.bias.detach().zero_()        
    def forward(self,x):
        y=self.linear1(x); y=self.linear1bn(y)
        y=tnnf.relu(y)
        y=self.linear2(y); y=self.linear2bn(y)
        y=tnnf.relu(y)
        y=self.linear3(y); y=self.linear3bn(y)
        y=tnnf.relu(y)
        logits=self.linear_out(y)
        probs=tnnf.log_softmax(logits,dim=1)
        return logits,probs
torch.manual_seed(random_seed)
model=MLPN2(num_features=num_features2,
            num_classes=num_classes2)
model=model.to(dev); learning_rate=.001
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

In [19]:
%train_run2 30

Epoch: 001/030 | Batch 000/177 | Cost: 4.0540
Epoch: 001/030 train accuracy: 4.06%
Epoch: 002/030 | Batch 000/177 | Cost: 3.6881
Epoch: 002/030 train accuracy: 4.98%
Epoch: 003/030 | Batch 000/177 | Cost: 3.6466
Epoch: 003/030 train accuracy: 6.20%
Epoch: 004/030 | Batch 000/177 | Cost: 3.5837
Epoch: 004/030 train accuracy: 7.39%
Epoch: 005/030 | Batch 000/177 | Cost: 3.2699
Epoch: 005/030 train accuracy: 8.09%
Epoch: 006/030 | Batch 000/177 | Cost: 3.4331
Epoch: 006/030 train accuracy: 9.35%
Epoch: 007/030 | Batch 000/177 | Cost: 3.4588
Epoch: 007/030 train accuracy: 10.62%
Epoch: 008/030 | Batch 000/177 | Cost: 3.1379
Epoch: 008/030 train accuracy: 11.22%
Epoch: 009/030 | Batch 000/177 | Cost: 3.3201
Epoch: 009/030 train accuracy: 12.53%
Epoch: 010/030 | Batch 000/177 | Cost: 3.1678
Epoch: 010/030 train accuracy: 13.66%
Epoch: 011/030 | Batch 000/177 | Cost: 3.0232
Epoch: 011/030 train accuracy: 14.62%
Epoch: 012/030 | Batch 000/177 | Cost: 3.0326
Epoch: 012/030 train accuracy: 15.48

In [20]:
%print_acc2 train
%print_acc2 test

Train accuracy: 28.4884%
Test accuracy: 23.6082%
